In [1]:
import time
import requests, re
from bs4 import BeautifulSoup

from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)

from datetime import datetime, timedelta

from tqdm import tqdm

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
headers = {
    'user-agent' : ua.random
}

In [7]:
#갤러리 타입 가져오기(마이너, 일반)
def get_gallery_type(dc_id):
    #url로 requests를 보내서 redirect시키는지 체크한다.
    url = f'https://gall.dcinside.com/board/lists/?id={dc_id}'
    result = url
    
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "lxml")
    if "location.replace" in str(soup):
        redirect_url = str(soup).split('"')[3]
        result = redirect_url
    if "mgallery" in result:
        result = "mgallery/board"
    else:
        result = "board"
        
    return result

In [5]:
def make_galleryURL(gall_id):
    base_url = "https://gall.dcinside.com/"
    url = base_url + get_gallery_type(gall_id) + "/lists?id=" + gall_id

    return url

In [9]:
# dnfqq = 지하와 용사 갤러리
# football_new8 = 해외 축구 갤러리
# comic_new3 = 만화 갤러리


print(make_galleryURL('dnfqq'))
print(make_galleryURL('football_new8'))
print(make_galleryURL('comic_new3'))

https://gall.dcinside.com/mgallery/board/lists?id=dnfqq
https://gall.dcinside.com/board/lists?id=football_new8
https://gall.dcinside.com/board/lists?id=comic_new3


In [ ]:
#글 파싱 함수
def article_parse(last_timestamp):
    
    params = { 'list_num': '100' }
    
    save_data = ['num', 'title', 'content', 'reply', 'refresh', 'recommand', 'timestamp', 'link']
    
    result = {key : [] for key in save_data}
    page = 1
    today = time.localtime(time.time())  # 크롤링 시작일
    
    custom_break = False
    proxy_idx = 0
    proxy_data =  get_proxy_list()
    
    y_day_string = str(last_timestamp.year) + "-" + str(last_timestamp.month) + "-" + str(last_timestamp.day)
    y_day = pd.to_datetime(y_day_string)

    while True:
        try:
            if proxy_idx >= len(proxy_data):
                proxy_data =  get_proxy_list()

    
            proxies = {'http' : 'http://' + proxy_data[proxy_idx][1], 'https' : 'http://' + proxy_data[proxy_idx][1]}
            headers = {
                        'user-agent' : ua.random
                    }

            url = f"https://gall.dcinside.com/mgallery/board/lists?id=dnfqq&page={page}"
            res = requests.get(url, headers=headers, params=params, proxies = proxies, timeout=(10,10))            
            soup = BeautifulSoup(res.text, "lxml")


            article_list = soup.select(".us-post") #글 박스 전부 select
            
            is_target = is_targetDay(article_list, y_day)
            if is_target == 0:
                print('FIND TARGET DATE POST')
            elif is_target == 1:
                print('Recent Post Not Target Data')
                page+=1
                continue
            elif is_target == 2:
                print('Find all Will Breaking Roup..')
                
            for element in article_list:
    #             print(element.select_one('.gall_subject').text)
                if element.select_one('.gall_subject').text != '일반' : continue # 일반글이 아니면 패스

                timestamp = element.select_one(".gall_date")['title'] # 작성시간

                if pd.to_datetime(timestamp) < last_timestamp:
                    print('Break post Date :',timestamp)
                    custom_break = True
                    break
                #글 박스를 하나씩 반복하면서 정보 추출
                link = "https://gall.dcinside.com" + element.select("a")[0]['href'].strip()
#                 content = get_post_content(link, proxy_data, proxy_idx)
                content = ''
#                 print(link, content)
                num = element.select(".gall_num")[0].text # 게시글 번호
                title = element.select(".ub-word > a")[0].text # 제목
                reply = element.select(".ub-word > a.reply_numbox > .reply_num") # 댓글 수
                if reply : reply = reply[0].text
                else : reply = ""
        #         nickname = element.select(".ub-writer")[0].text.strip() # 게시자 닉네임
                refresh = element.select(".gall_count")[0].text # 조회수
                recommand = element.select(".gall_recommend")[0].text # 추천수

                page_data = [num, title, content, reply, refresh, recommand, timestamp, link]

                for key, data in zip(result.keys(), page_data):
                    result[key].append(data)

            proxy_idx = 0
            if custom_break:
                is_today = False
            else:
                print('Data CNT : ', len(result['timestamp']))
                print('custom_break : False - ',result['timestamp'][-1])
                is_today = pd.to_datetime(result['timestamp'][-1]) >= y_day

            if not is_today : 
                print(page)
                break
            else:
                page+=1


        except Exception as e:
            print('Error! : ', e)
            proxy_idx+=1
            if proxy_idx > len(proxy_data):
                proxy_idx = 0
            pass
        
    return result

In [81]:
def get_post_content(link, proxy_data, proxy_idx):
    
    while True:
        try:
            proxies = {'http' : 'http://' + proxy_data[proxy_idx][1], 'https' : 'http://' + proxy_data[proxy_idx][1]}
            headers = {
                    'user-agent' : ua.random
                }
            
#             res = requests.get(link, headers=headers, proxies = proxies, timeout=(5,10))
            res = requests.get(link, headers=headers, proxies = proxies, timeout=(10,10))
            if res.text != '':
                break
        except Exception as e:
            proxy_idx +=1
            
            if proxy_idx > len(proxy_data):
                proxy_idx = 0
            pass
        
    soup = BeautifulSoup(res.text, "lxml")
    
    is_content = soup.select_one('.write_div')
    
    if is_content : content = is_content.text
    else : content = ''
    
    return content, proxy_idx

# 댓글 부분을 못가져와서 일단 보류
#     reply_list = soup.select('.comment_count')
#     replies = []
#     for element in reply_list:
#         reply = element.select_one('div.clear.cmt_txtbox.btn_reply_write_all > p').text
#         replies.append(reply)
    
    
#     return content, replies

In [12]:
def get_proxy_list():
    url = 'http://spys.one/en/free-proxy-list/'
    data = {
        'xpp' : '1',
        'xf1' : '0',
        'xf2' : '0',
        'xf4' : '0',
        'xf5' : '2'
    }

    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
    }

    r = requests.post(url, data=data, headers=headers)

    soup = BeautifulSoup(r.content, 'html.parser')

    result = []

    ports = {}
    script = soup.select_one('body > script')
    for row in script.text.split(';'):
        if '^' in row:
            line = row.split('=')
            ports[line[0]]  = line[1].split('^')[0]

    trs = soup.select('tr[onmouseover]')
    for tr in trs:

        is_http = tr.select_one('a')

        if is_http is not None:
            http = is_http.text
#             print(is_http.text)

        e_ip = tr.select_one('font.spy14')
#         print(e_ip)
        ip = ''

        e_port = tr.select_one('script')
        port = ''
        if e_port is not None:
            re_port = re.compile(r'\(([a-zA-Z0-9]+)\^[a-zA-Z0-9]+\)')
            match = re_port.findall(e_port.text)
            for item in match:
                port = port + ports[item]
        else:
            continue

        if e_ip is not None:
            for item in e_ip.findAll('script'):
                item.extract()
            ip = e_ip.text

        else:
            continue

        tds = tr.select("td")
        is_skip = False
        for td in tds:
            e_pct = td.select_one('font > acronym')
            if e_pct is not None:
                pct = re.sub('([0-9]+)%.*', r'\1', e_pct.text)
                if not pct.isdigit():
                    is_skip = True
            else:
                continue

        if not pct.isdigit():
            continue

        result.append((http,ip + ':' + port, pct))

    result.sort(key = lambda element  : int(element[2]), reverse=True)

    return result

In [94]:
#글 파싱 함수
def get_gallery_post(gallery_id, end_date = ''):
    
    params = { 'list_num': '30' }
    
    proxy_data =  get_proxy_list()
    proxy_idx = 0
    page = 1
    break_flag = False
    
    while True:
        try:

            proxies = {'http' : 'http://' + proxy_data[proxy_idx][1], 'https' : 'http://' + proxy_data[proxy_idx][1]}
            headers = {
                        'user-agent' : ua.random
                    }

            url = make_galleryURL(gallery_id)
            res = requests.get(url, headers=headers, params = params ,proxies = proxies, timeout=(10,10))            
            soup = BeautifulSoup(res.text, "lxml")

            break_flag = True

            if break_flag:
                break

        except Exception as e:
            print('Error! : ', e)
            proxy_idx+=1
            if proxy_idx > len(proxy_data):
                proxy_idx = 0
            pass
        
    return soup

In [95]:
soup = get_gallery_post('football_new8')

Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /board/lists?id=football_new8&list_num=30 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /board/lists?id=football_new8&list_num=30 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /board/lists?id=football_new8&list_num=30 (Caused by ProxyError('Cannot connect to proxy.', timeout('_ssl.c:1074: The handshake operation timed out')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /board/lists?id=football_new8&list_num=30 (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 500 Internal Server Error')))
Error

In [96]:
posts = soup.select(".us-post") #글 박스 전부 select

In [97]:
num_lst, title_lst, link_lst = [], [], []

content_lst = []

for idx,element in enumerate(posts):
    if element.select_one('.gall_subject'):
        if element.select_one('.gall_subject').text == '공지' : continue # 공지글이면 패스

    timestamp = element.select_one(".gall_date")['title'] # 작성시간

    #글 박스를 하나씩 반복하면서 정보 추출
    link = "https://gall.dcinside.com" + element.select("a")[0]['href'].strip()

    content, proxy_idx = get_post_content(link, proxy_data, proxy_idx)

    num = element.select(".gall_num")[0].text # 게시글 번호
    title = element.select(".ub-word > a")[0].text # 제목

    refresh = element.select(".gall_count")[0].text # 조회수
    recommand = element.select(".gall_recommend")[0].text # 추천수
    
    num_lst.append(num)
    title_lst.append(title)
    link_lst.append(link)
    content_lst.append(content)

12
HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /board/view/?id=football_new8&no=3616341&list_num=30&page=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f94b9f22f50>, 'Connection to 115.127.94.18 timed out. (connect timeout=10)'))
13
HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /board/view/?id=football_new8&no=3616341&list_num=30&page=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f94b8abba10>, 'Connection to 79.174.188.153 timed out. (connect timeout=10)'))
14
Find Content!
14
HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /board/view/?id=football_new8&no=3616340&list_num=30&page=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f94b9448810>, 'Connection to 177.93.36.150 timed out. (connect timeout=10)'))
15
HTTPSConnectionPool(host='gall.dcinside.com', port=4

In [98]:
df = pd.DataFrame({
    'num' : num_lst,
    'title' : title_lst,
    'content' : content_lst,
    'link' : link_lst
})

In [99]:
df.head()

,num,title,content,link
0,3616341,일단 분데스라는 리그자체는 올려치기 힘들죠,\n딴리그 전부 38경기하면서 헐떡댈때 휴식기 1달이나 여유있게 갖고서 양학하는곳이...,https://gall.dcinside.com/board/view/?id=footb...
1,3616340,이강인 파리는 좀 무리였던거 같은디,\n파리 이번시즌 걍 돈지랄로 무지성영입 존나해서 스쿼드에 선수가 너무 많음... ...,https://gall.dcinside.com/board/view/?id=footb...
2,3616339,신두형 비하하는 디시콘 제작하면 지속적으로 산고 합니다.,\n그렇게 아세요. \n,https://gall.dcinside.com/board/view/?id=footb...
3,3616338,긴급속보) 메시가 이강인 맞팔해줌ㅋㅋㅋㅆㅂ실화냐이거,\n사랑해 리나야\n,https://gall.dcinside.com/board/view/?id=footb...
4,3616336,자리가 사람을 만드는거야,\n벤제마가 인증했자나 다늙은 크카모 지원받고도 그자리 맡으니까 챔스우승 발롱개좆두...,https://gall.dcinside.com/board/view/?id=footb...


In [116]:
df['link'].iloc[1]

'https://gall.dcinside.com/board/view/?id=football_new8&no=3616340&list_num=30&page=1'

In [121]:
res = requests.get(df['link'].iloc[1], headers=headers)

In [124]:
soup = BeautifulSoup(res.text, 'html.parser')

In [129]:
from lxml import etree

In [130]:
dom = etree.HTML(str(soup))

In [135]:
dom.xpath('//*[@id="container"]/section/article[2]/div[1]/header/div/h3/span[2]')[0].text

'이강인 파리는 좀 무리였던거 같은디'